在本教程中，你将了解什么是**类别变量（categorical variable）**，以及三种处理这类数据的方法。


# 引言

**类别变量**只取有限个离散取值。

- 例如，一个关于早餐频率的问卷提供四个选项：“从不（Never）”“很少（Rarely）”“大多数天（Most days）”“每天（Every day）”。由于回答只落在固定的几个类别中，这就是类别数据。
- 如果让受访者填写所拥有的汽车品牌，答案会是 “Honda”“Toyota”“Ford”等类别，这同样是类别数据。

如果不先进行预处理，直接把这些变量塞进大多数 Python 中的机器学习模型会报错。本教程将比较三种可用于准备类别数据的方法。

# 三种方法

### 1）删除类别变量

处理类别变量最简单的办法就是直接把它们从数据集中移除。此法仅在这些列不包含有用信息时才可能工作得较好。

### 2）序数编码（Ordinal Encoding）

**序数编码**为每个唯一取值分配一个不同的整数。

![tut3_ordinalencode](https://storage.googleapis.com/kaggle-media/learn/images/tEogUAr.png)

这种方法隐含了类别的顺序关系：“Never (0) < Rarely (1) < Most days (2) < Every day (3)”。

在该示例中这一假设是合理的，因为这些类别存在明确的大小关系。并非所有类别变量都有清晰的顺序，我们把有顺序的称为**序数变量（ordinal）**。对于基于树的模型（如决策树与随机森林），序数编码通常能很好地适配序数变量。

### 3）独热编码（One-Hot Encoding）

**独热编码**会创建新列来指示原始数据中每个可能取值的存在（或不存在）。我们通过一个例子来理解：

![tut3_onehot](https://storage.googleapis.com/kaggle-media/learn/images/TW5m0aJ.png)

在原始数据中，“Color” 是一个包含三个类别（Red、Yellow、Green）的类别变量。相应的独热编码包含每个可能取值的一列，以及与原始数据等长的行数。原值为 “Red” 的地方，在 “Red” 列写入 1；原值为 “Yellow” 的地方，在 “Yellow” 列写入 1，依此类推。

与序数编码不同，独热编码并不假设类别存在顺序。因此，当类别数据没有明确次序（例如 Red 既不是“比 Yellow 更大”，也不是“更小”）时，独热编码尤其适用。我们把没有内在排序的类别变量称为**名义变量（nominal）**。

当某个类别变量的取值种类非常多时（通常超过 15 种），独热编码的表现往往不好。

# 示例

与前一节一样，我们仍使用[墨尔本房价数据集](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot/home)。

我们不展开数据加载细节。你可以假设我们已经在 `X_train`、`X_valid`、`y_train`、`y_valid` 中准备好了训练/验证数据。

In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv('../input/melbourne-housing-snapshot/melb_data.csv')

# Separate target from predictors
y = data.Price
X = data.drop(['Price'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# Drop columns with missing values (simplest approach)
cols_with_missing = [col for col in X_train_full.columns if X_train_full[col].isnull().any()] 
X_train_full.drop(cols_with_missing, axis=1, inplace=True)
X_valid_full.drop(cols_with_missing, axis=1, inplace=True)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

我们使用 `head()` 方法快速查看训练数据。

In [ ]:
X_train.head()

接下来，我们获取训练数据中所有类别变量的列表。

具体做法是检查每列的数据类型（**dtype**）。`object` 类型表示该列包含文本（理论上也可能是其他类型，但对我们来说不重要）。在本数据集中，文本列就是类别变量。

In [ ]:
# Get list of categorical variables
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

### 定义用于评估各方法质量的函数

我们定义函数 `score_dataset()` 来比较三种类别变量处理方法。该函数返回随机森林模型的[平均绝对误差](https://en.wikipedia.org/wiki/Mean_absolute_error)（MAE）。一般而言，MAE 越低越好！

In [ ]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

### 方法一评分（删除类别变量）

我们使用 [`select_dtypes()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.select_dtypes.html) 方法删除 `object` 类型的列。

In [ ]:
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

### 方法二评分（序数编码）

Scikit-learn 提供了 [`OrdinalEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html) 类来进行序数编码。我们遍历所有类别变量，并对每一列单独应用该编码器。

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# Make copy to avoid changing original data 
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# Apply ordinal encoder to each column with categorical data
ordinal_encoder = OrdinalEncoder()
label_X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])
label_X_valid[object_cols] = ordinal_encoder.transform(X_valid[object_cols])

print("MAE from Approach 2 (Ordinal Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

在上面的代码单元中，我们为每一列将每个唯一取值随机映射到一个整数。这是一种常见且比自定义标签更简单的做法；不过，如果我们能为所有序数变量提供更符合语义的标签，往往还能进一步提升性能。

### 方法三评分（独热编码）

我们使用 scikit-learn 的 [`OneHotEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) 来获得独热编码。该类有多个参数可用来定制行为：
- 将 `handle_unknown='ignore'`，以避免验证集中出现训练集未见类别时触发错误；
- 设置 `sparse=False`，以确保返回的是 numpy 数组（而非稀疏矩阵）。

在使用编码器时，我们只传入需要做独热编码的类别列。例如对训练集进行编码时，传入 `X_train[object_cols]`。（下面代码中的 `object_cols` 是包含类别列列名的列表，因此 `X_train[object_cols]` 就包含了训练集中所有的类别数据。）

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

# Ensure all columns have string type
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)

print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

# 哪种方法更好？

在这个例子中，删除类别列（**方法一**）表现最差，因为其 MAE 最高。对于另外两种方法，由于返回的 MAE 值非常接近，难以说哪一种具有显著优势。

一般而言，独热编码（**方法三**）通常表现最好，而删除类别列（**方法一**）通常表现最差，但具体仍取决于数据集。

# 结论

现实世界充满了类别数据。掌握如何处理这种常见数据类型，会让你成为更高效的数据科学家！

# 轮到你了

在 **[下一道练习](https://www.kaggle.com/kernels/fork/3370279)** 中应用你刚学到的技巧吧！

---




*有问题或想交流？欢迎访问[课程讨论区](https://www.kaggle.com/learn/intermediate-machine-learning/discussion)与其他学习者交流。*